# Contenidos

1. [Introducción](#intro)  
2. [Regresión](#pp)  
3. [Clasificación](#pdSns)  
4. [Clustering](#conclu)

# 1. Introducción



**¿Qué es el Aprendizaje de Máquinas?**

Instalar Plotly

# 2. Regresión

El problema de regresión consiste en encontrar una relación entre dos variables: entrada y salida, estímulo y respuesta, instancia y etiqueta, etc. El caso más simple, y que sirve de ilustración para modelos más expresivos es el de regresión lineal explorada en la tarea 1. En este módulo, se explora una manera de enriquecer la hipótesis de linealidad en los datos para obtener modelos más expresivos.

**Ejercicios**

* Cargue los datos presentes en `datos/C4` con el nombre de `aeropuerto.txt`. (Hint:pd.read_fwf)

* Separe los datos de manera temporal, para ello almacene el 75% de los datos iniciales en la variable `data_train` y el 25% restante en la variable `data_val`. 

Para la segunda pregunta, es posible hacer la selección de registros al azar usando las funcionalidades de Pandas, sin embargo, la manera **directa** de hacerlo es usando la librería `scikit-learn`, que corresponde al estándar de uso en la comunidad de machine learning (símil de Numpy en cuando a manejo de arreglos), en este capitulo y en particular en esta sub-sección, se estudian de manera práctica las funcionalidades que esta librería ofrece.

In [117]:
import sklearn as sk # importación de la libreria (global)

# Importación de la función especifica de "split"
from sklearn.model_selection import train_test_split 

En el caso anterior, la opción `shuffle` permite indicar si se desea particionar los datos de manera aleatoria o secuencial.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

data = pd.read_fwf('datos/C4/aeropuerto.txt', header = None, index_col=0)
data.columns    = ['Pasajeros']
data.index.name = 'Mes'

data_train, data_val = train_test_split(data,test_size=0.25, random_state=1, shuffle=False)

Una vez cargados y separados los datos, es necesario visualizarlos:

**Ejercicio**

* Grafíque los datos, de manera que los datos de entrenamiento y validación posean colores distintos.

In [ ]:
fig1 = plt.subplots()
plt.title('Pasajeros medidos por Mes')

plt.scatter(x = data_train.index,y = data_train.Pasajeros)
plt.scatter(x = data_val.index,y = data_val.Pasajeros,color ='r')

plt.xlabel('Mes'); plt.ylabel('Pasajeros')
plt.legend(['Train','Validacion'])

Para ajustar una regresión lineal en los datos se utiliza la función `linear_model` del módulo `sklearn`:

In [ ]:
from sklearn import linear_model

# Se inicializa el modelo
regr = linear_model.LinearRegression()

np_f = lambda x: np.array(x).reshape([-1,1])

# Se entrena el modelo
x_train = np_f(data_train.index)
y_train = np_f(data_train.Pasajeros)

regr.fit(x_train,y_train)
regr.score(x_train,y_train)

# Puntos de vlaidación
x_val = np_f(data_val.index)
y_val = np_f(data_val.Pasajeros)

# Se predice con el modelo
y_pred = regr.predict(x_val)

print(f'Error cuadrático medio es {sk.metrics.mean_squared_error(y_pred,y_val):.3f}')

Se visualizan los resultados

In [ ]:
ax, fig = plt.subplots()

plt.scatter(x_val,y_val,c='r')
plt.plot(x_val,y_pred, 'k--', label='regresión')

fig.set_title('Predicción en Validación')
fig.legend()

 **Ejercicio**
 
 * Grafíque la línea de regresión en todo el dataset.

In [ ]:
ax1, fig1 = plt.subplots()
x = np_f(data.index)

fig1.set_title('Predicción en el dataset')

y_pred = regr.predict(x)

plt.scatter(x = data_train.index,y = data_train.Pasajeros, label='Train')
plt.scatter(x = data_val.index,y = data_val.Pasajeros, color ='r', label='Validación')

plt.plot(data.index,y_pred, 'k--', label='regresión')

fig1.legend()

Como es posible observar, la regresión lineal descubre en cierto grado la *tendencia* de los datos a crecer, sin embargo, si se deseara predecir el comportamiento futuro de la serie de datos, la regresión lineal no sería capaz de proporcionar mayor certeza. Una manera de enriquecer este proceso es transformando los datos u *obteniendo características* de estos. Esta idea da lugar al concepto de regresión no lineal y consiste en modificar los datos iniciales a través de una transformación para luego hacer regresión sobre los "nuevos" datos o "características" obtenidos.

**Ejercicio**

* Utilice `PolynomialFeatures` del módulo `sklearn.preprocessing` para transformar todos los datos de entrada (Mes) a para grados del 1 al 5.

* Use todos datos (transformados) para calcular una nueva regresión lineal y compare sus rendimientos. (`sk.metrics.mean_squared_error`)

* ¿Qué ocurre cuando se seleccionan los conjuntos de entrenamiento y validación al azar?

* Grafíque los resultados obtenidos.

In [ ]:
from sklearn.preprocessing import PolynomialFeatures

mods = []
for g in np.arange(1,6):
    
    poly = PolynomialFeatures(degree=g)
    
    X = poly.fit_transform(x_train)
    x_val_poly = poly.fit_transform(x_val)
    
    poly_regr = linear_model.LinearRegression()
    poly_regr.fit(X, y_train)
    
    y_pred_poly = poly_regr.predict(x_val_poly)
    mods.append(poly_regr)
    
    print(f'Error cuadrático porlinomio grado {g} es {sk.metrics.mean_squared_error(y_pred_poly,y_val)}')
    
# Grado Seleccionado
g=2

ax2, fig2 = plt.subplots()
fig2.set_title('Predicción en el dataset con g={}'.format(g))

poly = PolynomialFeatures(degree=g)
X = poly.fit_transform(x)
poly_regr = mods[g-1]

y_pred_poly = poly_regr.predict(X)

plt.scatter(x = data_train.index,y = data_train.Pasajeros, label='entrenamiento')
plt.scatter(x = data_val.index,y = data_val.Pasajeros, color ='r', label='validación')

plt.plot(data.index,y_pred_poly, 'k--', label='regresión')

fig2.legend()


Como se pudo ver, la predicción en la tendencia se acerca más a la intuición al escoger una transformación en los datos. La técnica antes implementada se conoce como regresión polinomial y proviene de la clase de regresión lineal generalizada. A continuación, se desea modelar ciertas características fuera de la tendencia del modelo. 

**Ejercicio**

* ¿ Qué componente se aprecia fuera de la tendencia ?

Para modelar las componentes periódicas del problema anterior, se hace una aproximación de los "residuos" que ocurren al predecir con el modelo, es decir, se resta a los datos la predicción del modelo encontrado y se aborda el dataset obtenido como un nuevo problema de regresión. Este proceso se puede efectuar las veces que sean necesarias hasta obtener resultados razonables. 

Para implementar lo anterior:

**Ejercicio**

* Genere un nuevo dataset consistente en los residuos del dataset inicial, obtenidos con el modelo de regresión polinomial de grado 2.

* Grafíque los residuos tanto para entrenamiento como para validación.

In [ ]:
y_pred_poly = poly_regr.predict(X)

x_train_poly = poly.fit_transform(x_train)
x_val_poly   = poly.fit_transform(x_val)

y_res_train = y_train - poly_regr.predict(x_train_poly)
y_res_val = y_val - poly_regr.predict(x_val_poly)

ax3, fig3 = plt.subplots()
fig3.set_title('Residuos')

poly = PolynomialFeatures(degree=g)
X = poly.fit_transform(x)
poly_regr = mods[g-1]

y_pred_poly = poly_regr.predict(X)

plt.scatter(x = data_train.index, y = y_res_train, label='Train')
plt.scatter(x = data_val.index,y = y_res_val, color ='r', label='Validación')
fig3.legend()

Se utilizará la siguiente transformación en los datos para modelar las componentes armónicas de los residuos:

In [ ]:
sin_comp  = lambda x,theta: theta[0]*np.sin(theta[1]*x +theta[2])*np.exp(theta[3]*x)

Para esta transformación no existe un método en `sklearn` por lo que habrá que aprender los parámetros por medio de optimización, para ello se utiliza la librería `scipy` y se define la función a minimizar:

In [ ]:
J = lambda theta : np.mean((y_res_train-sin_comp(x_train,theta))**2)

Se define un punto inicial:

In [ ]:
from scipy.optimize import minimize

theta_0 = np.array([1, 0.3, 1.6 , 0.01])

h = minimize(fun = J, x0 = theta_0, method='BFGS')

theta = h.x
print(theta)

Y se calculan las nuevas predicciones:

In [ ]:
y_pred_sin = sin_comp(x_train,theta)

**Ejercicio**

* Grafíque la aproximación a los residuos.
* Obtenga la función que aproxima todo el dataset y compárela con la estimación de la tendencia que se hizo inicialmente.

In [ ]:
ax4, fig4 = plt.subplots()
fig4.set_title('Residuos aproximados')

plt.scatter(x = data_train.index, y = y_res_train, label='Train')
plt.scatter(x = data_val.index,y = y_res_val, color ='r', label='Validación')
plt.plot(y_pred_sin)
fig4.legend()

In [ ]:
f_pol_sin = lambda x: poly_regr.predict(poly.fit_transform(x)) + sin_comp(x,theta=theta)
y_pred_final = f_pol_sin(x)

plt.scatter(x = data_train.index, y = y_train, label='Train')
plt.scatter(x = data_val.index,y = y_val, color ='r', label='Validación')
plt.plot(y_pred_final, 'k--', label='Predicción')
plt.legend()


El método de regresión jerárquica por residuos ofrece gran flexibilidad a la hora de enfrentar tareas de regresión. Como se pudo ver en este módulo, es posible incluso implementar transformaciones de datos no incluidas en los paquetes tradicionales. Se recomienda explorar los otros métodos de regresión como son los procesos gaussianos y support vector machines presentes en `sklearn`.

# 3. Clasificación 

El problema de clasificación, corresponde al igual que el de regresión, a encontrar una relación entre entradas y salidas de datos, la diferencia entre ambos problema radica en que el problema de clasificación consiste en asignar valores categóricos o discretos mientras que en regresión estos son continuos. A continuación se estudian métodos de _machine learning_ aplicados en este contexto.

In [ ]:
# Modelos de clasificación estudiados
from sklearn.linear_model import LogisticRegression #regresión logística
from sklearn.svm import SVC, LinearSVC #support vector machines
from sklearn.ensemble import RandomForestClassifier #random forests
from sklearn.neighbors import KNeighborsClassifier #K-nearest neighbors

**Ejercicio**

* Cargue los datos presentes en `datos/C4` con el nombre de `titanic.csv`. (Hint:pd.read_csv)
* Separe los datos de manera aleatoria, para ello almacene el 75% de los datos iniciales en la variable train_df y el 25% restante en la variable val_df.

* Describa los datos importados. ¿Qué columnas contienen valores nulos o mal ingresados?

Observación: `desribe(include=['O'])`permite estudiar las variables categóricas.

In [ ]:
df = pd.read_csv('datos/C4/titanic.csv')

In [ ]:
train_df , val_df = train_test_split(df,test_size=0.25, random_state=1, shuffle=True)
train_df.head()

In [ ]:
val_df.columns

In [ ]:
train_df.info()

In [ ]:
train_df.describe(include=['O'])

### Análisis de los datos

**Datos Faltantes**

In [ ]:
valores_faltantes = pd.concat([train_df.isnull().sum(), val_df.isnull().sum()],
                              axis=1, sort=True,keys=['Train', 'Val']) 
valores_faltantes

**Agrupaciones de interés**

In [ ]:
print(train_df[['Pclass', 'Survived']].groupby(['Pclass'], as_index=False).mean().sort_values(by='Survived', ascending=False))

In [ ]:
print(train_df[["Sex", "Survived"]].groupby(['Sex'], as_index=False).mean().sort_values(by='Survived', ascending=False))

### Limpieza de datos

**Age**

Se agregan edades aleatorias dentro del promedio existente:

In [ ]:
new_ages_train = np.random.randint(train_df["Age"].mean() - train_df["Age"].std(),
                                   train_df["Age"].mean() + train_df["Age"].std(),
                                    size = train_df["Age"].isnull().sum())

new_ages_val  = np.random.randint(val_df["Age"].mean() - val_df["Age"].std(),
                                  val_df["Age"].mean() + val_df["Age"].std(),
                                  size = val_df["Age"].isnull().sum())

train_df['Age'][train_df["Age"].isnull()] = new_ages_train
val_df["Age"][val_df["Age"].isnull()]   = new_ages_val

train_df['Age'] = train_df['Age'].astype(int)
val_df['Age']  = val_df['Age'].astype(int)

**Embarked y Port**

Se agrega la categoria 'S' y se mapea con índices discretos. 

In [ ]:
train_df["Embarked"].fillna('S', inplace=True)
val_df["Embarked"].fillna('S', inplace=True)

train_df['Port'] = train_df['Embarked'].map( {'S': 0, 'C': 1, 'Q': 2} ).astype(int)
val_df['Port'] = val_df['Embarked'].map({'S': 0, 'C': 1, 'Q': 2}).astype(int)

del train_df['Embarked']
del val_df['Embarked']

**Cabin**

Se elimina por poseer demasiados valores faltantes.

In [ ]:
del train_df['Cabin']
del val_df['Cabin']

**Name y Ticket**

Se eliminan por su poca relevancia.

In [ ]:
del train_df['Name']
del train_df['Ticket']

del val_df['Name']
del val_df['Ticket']

**Sex**

Se traspasa a variable discreta.

In [ ]:
train_df = pd.get_dummies(train_df,columns=['Sex'])
val_df   = pd.get_dummies(val_df,columns=['Sex'])

### Regresión logística

In [ ]:
X_train = train_df.drop("Survived", axis=1)
Y_train = train_df["Survived"]

X_val  = val_df.drop("PassengerId", axis=1).copy()
print('Dimensiones de los datos son:')
X_train.shape, Y_train.shape, X_val.shape

In [ ]:
logreg = LogisticRegression()
logreg.fit(X_train, Y_train)

Y_pred = logreg.predict(X_val)
acc_log = round(logreg.score(X_train, Y_train) * 100, 2)
print(f'Precisión de regresión logística es {acc_log} (en entrenamiento)')

### Suport Vector Machines

In [ ]:
svc = SVC()
svc.fit(X_train, Y_train)

Y_pred = svc.predict(X_val)
acc_svc = round(svc.score(X_train, Y_train) * 100, 2)
print(f'Precisión de support vector machines es {acc_svc}  (en entrenamiento)')

## Random Forest

In [ ]:
random_forest = RandomForestClassifier(n_estimators=100)
random_forest.fit(X_train, Y_train)

Y_pred = random_forest.predict(X_val)
random_forest.score(X_train, Y_train)

acc_random_forest = round(random_forest.score(X_train, Y_train) * 100, 2)
print(f'Precisión de random forests es {acc_random_forest}  (en entrenamiento)')

## KNN

In [ ]:
knn = KNeighborsClassifier(n_neighbors = 3)
knn.fit(X_train, Y_train)

Y_pred = knn.predict(X_val)
acc_knn = round(knn.score(X_train, Y_train) * 100, 2)
print(f'Precisión de k nearest neighbors es {acc_knn} (en entrenamiento)')

## Resumen de modelos

In [ ]:
models = pd.DataFrame({
    'Model': ['Support Vector Machines', 'KNN', 'Logistic Regression', 
              'Random Forest'],
    'Score': [acc_svc, acc_knn, acc_log, 
              acc_random_forest]})
models.sort_values(by='Score', ascending=False)
print('agreguemos validación :)')

# 4. Clustering
## Encuesta de felicidad

El proceso de clustering o aglomeración consiste en agrupar una serie de vectores o datos según cierto criterio, éste por lo general se basa en distancia o similitud. 

Como se vio en el Capítulo 1, se podía definir un algoritmo de aglomeración según la distancia euclidiana, que se generalizó a la distancia $p$. En general, se busca que la métrica de aglomeración en conjunción con el modelo que se implementa, permitan obtener información no trivial sobre los datos que se estudian, así como también obtener representaciones o etiquetas de estos de manera *no supervisada*.

Siguiendo el esquema de los módulos anteriores, se observan algunos modelos de clustering:

**Ejercicio**

* Cargue los datos presentes en datos/C4 con el nombre de felicidad_2017.csv. (Hint:pd.read_csv)

In [ ]:
df = pd.read_csv('datos/C4/felicidad_2017.csv', header=0)
df.head(10)

* Ejecute la siguiente orden.¿Qué patrones pudo observar?

In [ ]:
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

init_notebook_mode(connected=True)

data = dict(type = 'choropleth', 
           locations = df['Country'],
           locationmode = 'country names',
           z = df['Happiness.Rank'], 
           text = df['Country'],
           colorbar = {'title':'Happiness'})

layout = dict(title = 'Global Happiness', 
             geo = dict(showframe = False, 
                       projection = {'type': 'natural earth'}))
choromap3 = go.Figure(data = [data], layout=layout)
iplot(choromap3)

* Use seaborn para investigar las correlaciones entre las variables.

In [ ]:
import seaborn as sns

fig=plt.figure()
sns.pairplot(df, diag_kind='kde')

Según lo anterior, se escogen 3 variables de interés dentro del dataset, estas se visualizan de la siguiente manera:

In [ ]:
ax, fig = plt.subplots()

fig.set_title('Ranking según índices seleccionados')

rank = df['Happiness.Rank']
plt.scatter(df['Economy..GDP.per.Capita.'],rank,label='Economy')
plt.scatter(df['Generosity'],rank,label='Generosity')
plt.scatter(df['Dystopia.Residual'],rank,label='Dys_res')
plt.ylabel('happiness ranking')
plt.xlabel('valor del índice')

plt.legend()

Usando el módulo de clustering de sklearn, se usarán distintos algoritmos de aglomeración y se observará su efecto en el mapa antes generado.

In [ ]:
# Se obtiene la lista de paises 
country = df.Country

# Se trabaja sobre los datos sin las columnas 'Country' y 'Happiness Rank'
data = df.drop(['Happiness.Rank','Country'], axis=1)
cols = data.columns 

# Se estandarizan los datos
from sklearn.preprocessing import StandardScaler

sc = StandardScaler()

data = pd.DataFrame(sc.fit_transform(data))
data.columns = cols

### K-Means

Este algoritmo ya fue estudiado en el Capítulo 1, ahora se usará la implementación que ofrece `sklearn`:

In [ ]:
from sklearn import cluster, mixture

k = 4

kmeans_alg = cluster.KMeans(k)
kmeans_res = kmeans_alg.fit_predict(data)

df_km = data.copy()

df_km['Kmeans']= kmeans_res

In [ ]:
df_km.head()

**Ejercicio**

* Grafíque los grupos encontrados para las variables 'Happiness.Score' , 'Dystopia.Residual'.¿Qué cantidad de clusters elige?

In [ ]:
plt.scatter(df_km['Happiness.Score'], df_km['Dystopia.Residual'],  c=kmeans_res)
plt.ylabel('happiness score ')
plt.xlabel('valor del índice')

* Agregue al dataframe `df_km` con los valores 'Country' del dataframe `data` y visualice con el siguiente comando:

In [ ]:
df_km['Country'] = country

In [ ]:
dataPlot = dict(type = 'choropleth', 
           locations = df_km['Country'],
           locationmode = 'country names',
           z = df_km['Kmeans'], 
           text = df_km['Country'],
           colorbar = {'title':'Cluster Group'})
layout = dict(title = 'Kmeans Clustering', 
           geo = dict(showframe = False, 
           projection = {'type': 'natural earth'}))
choromap3 = go.Figure(data = [dataPlot], layout=layout)
iplot(choromap3) 

### Clustering espectral

**Ejercicio** 

* Aplique la metodología anterior para la función `cluster.SpectralClustering`.

In [ ]:
k=4

spectral = cluster.SpectralClustering(n_clusters=k)
sp_res= spectral.fit_predict(data)

df_sp = data.copy()
df_sp['spectral'] = pd.DataFrame(sp_res)
df_sp['Country'] = country

In [ ]:
plt.scatter(df_km['Happiness.Score'], df_km['Dystopia.Residual'],  c= sp_res)
plt.ylabel('happiness score ')
plt.xlabel('valor del índice')

In [ ]:
dataPlot = dict(type = 'choropleth', 
           locations = df_sp['Country'],
           locationmode = 'country names',
           z = df_sp['spectral'], 
           text = df_sp['Country'],
           colorbar = {'title':'Cluster Group'})
layout = dict(title = 'Spectral Clustering', 
           geo = dict(showframe = False, 
           projection = {'type': 'natural earth'}))
choromap3 = go.Figure(data = [dataPlot], layout=layout)
iplot(choromap3) 

### DBSCAN

In [ ]:
eps = 1.35

db     = cluster.DBSCAN(eps)
db_res = db.fit_predict(data)

df_db = data.copy()
df_db['DBSCAN'] = pd.DataFrame(sp_res)
df_db['Country'] = country

plt.scatter(df_db['Happiness.Score'], df_db['Dystopia.Residual'],  c= db_res)

In [ ]:
dataPlot = dict(type = 'choropleth', 
           locations = df_sp['Country'],
           locationmode = 'country names',
           z = df_db['DBSCAN'], 
           text = df_db['Country'],
           colorbar = {'title':'Cluster Group'})
layout = dict(title = 'DBSCAN', 
           geo = dict(showframe = False, 
           projection = {'type': 'natural earth'}))
choromap3 = go.Figure(data = [dataPlot], layout=layout)
iplot(choromap3) 

### Gaussian Mixture Clustering

In [ ]:
comps = 4

gmm = mixture.GaussianMixture(n_components=comps,covariance_type='full')

gmm.fit(data)
gmm_res =gmm.predict(data)


df_gmm = data.copy()
df_gmm['GMM'] = pd.DataFrame(gmm_res)
df_gmm['Country'] = country

plt.scatter(df_gmm['Happiness.Score'], df_gmm['Dystopia.Residual'],  c= gmm_res)
plt.ylabel('happiness score ')
plt.xlabel('valor del índice')

In [ ]:
dataset=pd.concat([data,country],axis=1)
dataPlot = dict(type = 'choropleth', 
           locations = df_gmm['Country'],
           locationmode = 'country names',
           z = df_gmm['GMM'], 
           text = dataset['Country'],
           colorbar = {'title':'Cluster Group'})
layout = dict(title = 'GMM', 
           geo = dict(showframe = False, 
           projection = {'type': 'natural earth'}))
choromap3 = go.Figure(data = [dataPlot], layout=layout)
iplot(choromap3) 